In [7]:
import os
import sys

# 自动检测环境
IS_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ
competition_name = 'titanic/'

if IS_KAGGLE:
    print("Running on Kaggle")
    DATA_PATH = '/kaggle/input/' + competition_name
    OUTPUT_PATH = '/kaggle/working/' + competition_name
    # 可能需要安装包
else:
    print("Running locally")
    DATA_PATH = './data'
    OUTPUT_PATH = './output'

Running locally


In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [9]:
# Add src directory to Python path and import model
sys.path.append('./src')
from model import simple_logr_scaler

In [10]:
train = pd.read_csv(DATA_PATH + '/train.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [11]:
df_train = train.copy()
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
df_train = df_train.drop(['Name', 'Ticket'], axis = 1)
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,71.2833,C85,C
2,3,1,3,female,26.0,0,0,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,53.1000,C123,S
4,5,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,30.0000,B42,S
888,889,0,3,female,28.0,1,2,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,30.0000,C148,C


In [12]:
df_train['cabin_info_null'] = df_train['Cabin'].isna()
df_train['cabin_info_null'] = df_train['cabin_info_null'].astype('int32')
df_train = df_train.drop(['Cabin'], axis = 1)
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cabin_info_null
0,1,0,3,male,22.0,1,0,7.2500,S,1
1,2,1,1,female,38.0,1,0,71.2833,C,0
2,3,1,3,female,26.0,0,0,7.9250,S,1
3,4,1,1,female,35.0,1,0,53.1000,S,0
4,5,0,3,male,35.0,0,0,8.0500,S,1
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,1
887,888,1,1,female,19.0,0,0,30.0000,S,0
888,889,0,3,female,28.0,1,2,23.4500,S,1
889,890,1,1,male,26.0,0,0,30.0000,C,0


In [13]:
df_train_d = pd.get_dummies(df_train, columns=['Sex', 'Embarked'], dtype='int32')
df_train_d = df_train_d.drop('Sex_female', axis=1)
df_train_d

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,cabin_info_null,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,1,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000,1,1,0,0,1
887,888,1,1,19.0,0,0,30.0000,0,0,0,0,1
888,889,0,3,28.0,1,2,23.4500,1,0,0,0,1
889,890,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [14]:
X_train = np.array(df_train_d.drop(['Survived', 'PassengerId'], axis = 1))
Y_train = np.array(df_train_d['Survived'])

In [15]:
slr = simple_logr_scaler(X_train)
Xs = slr.fit()
Xs.shape
# Xs

(891, 10)

In [16]:
Xs_train = Xs[:800, :]
Xs_test = Xs[800:, :]
Ys_train = Y_train[:800]
Ys_test = Y_train[800:]

In [17]:
lrsr = log_R_solver(Xs_train, Ys_train, alpha = 0.03, lambda2=0.1)
lrsr.fit_L2()

NameError: name 'log_R_solver' is not defined

In [ ]:
Yr = lrsr.transfrom(Xs_train)

In [ ]:
np.sum(Ys_train == Yr) / Ys_train.shape[0]

0.79875

In [ ]:
Yrtst = lrsr.transfrom(Xs_test)
np.sum(Ys_test == Yrtst) / Ys_test.shape[0]

0.8131868131868132

In [ ]:
df_test = pd.read_csv(DATA_PATH + '/test.csv')
df_test['Embarked'] = df_train['Embarked'].fillna('S')
df_test['Age'] = df_test['Age'].fillna(df_train['Age'].median())
df_test = df_test.drop(['Name', 'Ticket'], axis = 1)
df_test['cabin_info_null'] = df_test['Cabin'].isna()
df_test['cabin_info_null'] = df_test['cabin_info_null'].astype('int32')
df_test = df_test.drop(['Cabin'], axis = 1)
df_test_d = pd.get_dummies(df_test, columns=['Sex', 'Embarked'], dtype='int32')
df_test_d = df_test_d.drop('Sex_female', axis=1)
X_test = np.array(df_test_d.drop(['PassengerId'], axis = 1))
Xtsts = slr.transform(X_test)
Yres = lrsr.transfrom(Xtsts)
df_test['Survived'] = Yres
df_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,cabin_info_null,Survived
0,892,3,male,34.5,0,0,7.8292,S,1,0
1,893,3,female,47.0,1,0,7.0000,C,1,0
2,894,2,male,62.0,0,0,9.6875,S,1,0
3,895,3,male,27.0,0,0,8.6625,S,1,0
4,896,3,female,22.0,1,1,12.2875,S,1,1
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,male,28.0,0,0,8.0500,S,1,0
414,1306,1,female,39.0,0,0,108.9000,S,0,1
415,1307,3,male,38.5,0,0,7.2500,S,1,0
416,1308,3,male,28.0,0,0,8.0500,S,1,0


In [ ]:
if IS_KAGGLE == False:
    os.makedirs('output', exist_ok=True)


In [ ]:
df_res = df_test[['PassengerId', 'Survived']]
df_res.to_csv(OUTPUT_PATH + '/v4_l2_reg.csv', index=False, encoding='utf-8')

In [ ]:
nslr = NN_solver(Xs_train, Ys_train, alpha=0.02, n_seed = 200, output_gap = 50)
# nslr.fit_batch_vectorized()
# nslr.fit_batch()
# nslr.fit_batch_adjust_lr(2000)
nslr.fit_shallow_parallel(shallow_iter_limit = 2000, target_loss = 320)

NameError: name 'NN_solver' is not defined